# PDPTW Benchmark Instances

Este notebook tem como objetivo estudar e analisar os dados extraidos do [Scopus](https://www.scopus.com/) dos artigos...

//TODO

In [1]:
import pandas as pd
import re
import matplotlib.pyplot as plt
plt.style.use('dark_background')
% matplotlib inline

In [2]:
# Importar dados dos arquivos, e concatena-los

arquivos = ["pdptw/li&lim(2003)/cite_li&lim(2003).csv"]

colunas_relevantes = ['Authors', 'Title', 'Year', 'Source title', 'Author Keywords', 'Index Keywords',
                      'Abstract', 'Cited by', 'Document Type', 'Language of Original Document']

def importar_csv_com_fonte(arquivo):
    df = (pd.read_csv(arquivo, usecols=colunas_relevantes)
            .assign(CiteArticle = re.search("/cite_(.*).csv", arquivo).group(1))
    )
    return (df)


artigos = (pd.concat(map(importar_csv_com_fonte, arquivos),)
           .reset_index(drop=True)
)

In [3]:
# Verificar se todos os dados foram corretamente importados
artigos.groupby("CiteArticle")["Title"].count()

CiteArticle
li&lim(2003)    142
Name: Title, dtype: int64

In [4]:
# Deletar dados duplicados
artigos.drop_duplicates(subset=colunas_relevantes, inplace=True)

Com os dados importados, procura-se filtrar os artigos que tratem de problemas dinâmicos. Para isso, será selecionado todo o artigo que mencionar a palavra "Dynamic" no título, resumo ou lista de palavras-chave

In [5]:
artigos_dinamicos = (artigos.assign(AllKeywords = lambda x: x["Author Keywords"].str.cat(x["Index Keywords"], na_rep=""))
                            .assign(DynamicTitle = lambda x: x.Title.str.contains("Dynamic"),
                                    DynamicKeywords = lambda x: x.AllKeywords.str.contains("Dynamic"),
                                    DynamicAbstract = lambda x: x.Abstract.str.contains("Dynamic"))
                            .assign(DynamicAny = lambda x: (x.DynamicTitle | x.DynamicKeywords | x.DynamicAbstract))                        
)
artigos_dinamicos = artigos_dinamicos.loc[artigos_dinamicos["DynamicAny"] == True,:]

In [6]:
artigos_dinamicos.shape

(19, 16)

In [7]:
artigos_dinamicos.groupby("Document Type")["Title"].count()

Document Type
Article             13
Conference Paper     5
Review               1
Name: Title, dtype: int64

Com esse filtro aplicado, sobraram ....

//TODO

In [8]:
artigos_dinamicos.groupby("Language of Original Document")["Title"].count()

Language of Original Document
English    19
Name: Title, dtype: int64

Lendo o resumo dos artigos restantes, foi possível determinar quais tratam de DARP, ou PDPTW dinâmicos e quais executam experimentos computacionais. Todos os artigos que não cumprem essas exigências foram deletados da análise.

In [9]:
artigos_para_deletar = ["An Incidental Delivery Based Method for Resolving Multirobot Pairwised Transportation Problems",
                        "Multi-agent models for transportation problems with different strategies of environment information propagation",
                        "Dynamic approach to solve the daily drayage problem with transit time uncertainty",
                        "Dynamic allocation of unmanned aerial vehicles with communication constraints",
                        "Restricted dynamic programming: A flexible framework for solving realistic VRPs",
                        "Modeling rich and dynamic vehicle routing problems in heuristiclab",
                        "Dynamic pickup and delivery with transfers",
                        "Vehicle routing problems with alternative paths: An application to on-demand transportation",
                        "Stopping and restarting strategy for stochastic sequential search in global optimization",
                        "Multi-agent environment for modelling and solving dynamic transport problems",
                        "Integration of vehicle routing and resource allocation in a dynamic logistics network",
                        "Rolling horizon scheduling algorithm for dynamic vehicle scheduling system",
                        "Multi-agent environment for dynamic transport planning and scheduling",                       
                       ]

In [10]:
artigos_dinamicos_filtrados = artigos_dinamicos.drop(artigos_dinamicos[artigos_dinamicos["Title"].isin(artigos_para_deletar)].index)

In [11]:
artigos_dinamicos_filtrados.iloc[2,1]

'Waiting and buffering strategies for the dynamic pickup and delivery problem with time windows'

Através da leitura das seções dos artigos que denotam as características dos experimentos, observa-se quais as instâncias usadas. A informação observada está inserida na coluna `Instances` no `DataFrame` `artigos_dinamicos`.

In [12]:
artigos_com_benchmark = artigos_dinamicos_filtrados.assign(Instance = False)
artigos_com_benchmark.loc[artigos_com_benchmark["Title"].str.contains('Measures of dynamism and urgency in logistics'),
                          "Instance"] = "Self-made"
artigos_com_benchmark.loc[artigos_com_benchmark["Title"].str.contains('Distribution of waiting time for dynamic pickup and delivery problems'),
                          "Instance"] = "Pankratz(2005)"
artigos_com_benchmark.loc[artigos_com_benchmark["Title"].str.contains('Waiting and buffering strategies for the dynamic pickup and delivery problem with time windows'),
                          "Instance"] = "Li&Lim(2001)"
artigos_com_benchmark.loc[artigos_com_benchmark["Title"].str.contains('Dynamic vehicle routing by means of a genetic algorithm'),
                          "Instance"] = "Pankratz(2005)"
artigos_com_benchmark.loc[artigos_com_benchmark["Title"].str.contains('A vehicle scheduler for on-demand bus systems based on a heuristic cost estimation'),
                          "Instance"] = "Self-made"

In [13]:
artigos_com_benchmark

,Authors,Title,Year,Source title,Cited by,Abstract,Author Keywords,Index Keywords,Language of Original Document,Document Type,CiteArticle,AllKeywords,DynamicTitle,DynamicKeywords,DynamicAbstract,DynamicAny,Instance
11,"Van Lon R.R.S., Ferrante E., Turgut A.E., Wens...",Measures of dynamism and urgency in logistics,2016,European Journal of Operational Research,3.0,Dynamism was originally defined as the proport...,Dynamism; Logistics; Measures; Transportation;...,Logistics; Mathematical models; Operations res...,English,Article,li&lim(2003),Dynamism; Logistics; Measures; Transportation;...,False,True,False,True,Self-made
25,"Vonolfen S., Affenzeller M.",Distribution of waiting time for dynamic picku...,2016,Annals of Operations Research,6.0,Pickup and delivery problems have numerous app...,Direct policy search; Dynamic pickup and deliv...,NaN,English,Article,li&lim(2003),Direct policy search; Dynamic pickup and deliv...,False,True,False,True,Pankratz(2005)
96,"Pureza V., Laporte G.",Waiting and buffering strategies for the dynam...,2008,INFOR,20.0,We investigate the impact of two strategies fo...,Dynamic pickup and delivery problems; Request ...,Buffering strategies; Conventional approaches;...,English,Article,li&lim(2003),Dynamic pickup and delivery problems; Request ...,False,True,False,True,Li&Lim(2001)
118,Pankratz G.,Dynamic vehicle routing by means of a genetic ...,2005,International Journal of Physical Distribution...,29.0,Purpose - To propose and to evaluate a new gen...,Production scheduling; Programming and algorit...,NaN,English,Article,li&lim(2003),Production scheduling; Programming and algorit...,True,False,False,True,Pankratz(2005)
138,"Fujita S., Nakatani A.",A vehicle scheduler for on-demand bus systems ...,2003,IEEE Conference on Intelligent Transportation ...,2.0,This paper proposes a scheduling scheme for so...,NaN,Costs; Dynamic programming; Intelligent system...,English,Conference Paper,li&lim(2003),Costs; Dynamic programming; Intelligent system...,False,True,False,True,Self-made
